In [1]:
# Explore safety issues in FDA 

In [2]:
import pandas as pd
from sodapy import Socrata
from sqlalchemy import create_engine, inspect
import sqlite3

In [3]:
client = Socrata("data.cityofchicago.org", None)


In [4]:
results = client.get("4ijn-s7e5", limit=100000)

In [5]:
results_df = pd.DataFrame.from_records(results)
results_df

,inspection_id,dba_name,aka_name,license_,facility_type,risk,address,city,state,zip,inspection_date,inspection_type,results,violations,latitude,longitude,location
0,2554087,Tropical Smoothie Cafe,Tropical Smoothie Cafe,2840460,Restaurant,Risk 2 (Medium),4874 N LINCOLN AVE,CHICAGO,IL,60625,2022-04-08T00:00:00.000,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.970550218072056,-87.68993650758067,"{'latitude': '-87.68993650758067', 'longitude'..."
1,2554077,MAYFAIR EARLY LEARNING,MAYFAIR EARLY LEARNING,2308804,Children's Services Facility,Risk 1 (High),4719 N PULASKI RD,CHICAGO,IL,60630,2022-04-08T00:00:00.000,License Re-Inspection,Pass,NaN,41.966991428127784,-87.72784541288395,"{'latitude': '-87.72784541288395', 'longitude'..."
2,2554079,MOCHINUT,MOCHINUT,2827751,Restaurant,Risk 2 (Medium),1139 W TAYLOR ST,CHICAGO,IL,60607,2022-04-08T00:00:00.000,License,Pass,36. THERMOMETERS PROVIDED & ACCURATE - Comment...,41.86932255085863,-87.65574774059874,"{'latitude': '-87.65574774059874', 'longitude'..."
3,2554086,GOLDEN STAR TOBACCO & GROCERY INC,GOLDEN STAR TOBACCO & GROCERY INC,2835873,Grocery Store,Risk 3 (Low),8637 S COTTAGE GROVE AVE,CHICAGO,IL,60619,2022-04-08T00:00:00.000,License,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.7373613466866,-87.60469580542522,"{'latitude': '-87.60469580542522', 'longitude'..."
4,2554099,PRESENCE CHICAGO HOSPITALS NETWORK,ST. JOSEPH HOSPITAL,1144381,Restaurant,Risk 1 (High),2900 N LAKE SHORE DR,CHICAGO,IL,60657,2022-04-08T00:00:00.000,Canvass,Fail,10. ADEQUATE HANDWASHING SINKS PROPERLY SUPPLI...,41.93440298707838,-87.63680583708285,"{'latitude': '-87.63680583708285', 'longitude'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1953816,NEW ENGLAND SEAFOOD COMPANY FL,NEW ENGLAND SEAFOOD COMPANY,2108692,Restaurant,Risk 1 (High),3341 N LINCOLN AVE,CHICAGO,IL,60657,2016-09-06T00:00:00.000,Canvass Re-Inspection,Pass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.942572824782836,-87.67051293219696,"{'latitude': '-87.67051293219696', 'longitude'..."
99996,1953835,CENTRO MARKET,EL CENTRO MARKET,66257,Grocery Store,Risk 1 (High),5153-59 S KEDZIE AVE,CHICAGO,IL,60632,2016-09-06T00:00:00.000,Canvass,Fail,12. HAND WASHING FACILITIES: WITH SOAP AND SAN...,41.799257043707684,-87.70357470655833,"{'latitude': '-87.70357470655833', 'longitude'..."
99997,1953693,CVS/PHARMACY #8753,CVS/PHARMACY #8753,1695423,Grocery Store,Risk 3 (Low),3637 N SOUTHPORT AVE,CHICAGO,IL,60613,2016-09-02T00:00:00.000,Canvass,Pass,"34. FLOORS: CONSTRUCTED PER CODE, CLEANED, GOO...",41.94817860023431,-87.66389687202293,"{'latitude': '-87.66389687202293', 'longitude'..."
99998,1953754,BAR TOMA,BAR TOMA,17495,Restaurant,Risk 1 (High),110 E PEARSON ST,CHICAGO,IL,60611,2016-09-02T00:00:00.000,Canvass,Pass,33. FOOD AND NON-FOOD CONTACT EQUIPMENT UTENSI...,41.89765954569349,-87.62553180957474,"{'latitude': '-87.62553180957474', 'longitude'..."


In [6]:
results_df=results_df.drop(columns= ["location"])

In [7]:
results_df=results_df[results_df["violations"].notna()]

In [8]:
engine = create_engine("sqlite:///inspections.db", echo=False)

# connect to the sql database
conn = sqlite3.connect('inspections.db')

# create the ability to perform logic on the database
c = conn.cursor()

# create the table in the database
c.execute('CREATE TABLE IF NOT EXISTS food_inspections '\
          '(inspection_id TEXT, dba_name TEXT, aka_name TEXT, license_ TEXT, facility_type TEXT, risk TEXT, address TEXT, '\
          'city TEXT, state TEXT, zip TEXT, inspection_date TEXT, inspection_type TEXT, results TEXT, violations TEXT, '\
          'R1_1 TEXT, R1_2 TEXT, R1_3 TEXT, R2_1 TEXT, R2_2 TEXT, R2_3 TEXT, RS_1 TEXT, RS_2 TEXT, RS_3 TEXT, '\
          'latitude TEXT, longituide TEXT, locations TEXT, '\
          'PRIMARY KEY (inspection_id))')
# commit the stock table
conn.commit()

# print table name
print(inspect(engine).get_table_names())

['food_inspections']


In [10]:
results_df.to_sql('food_inspections', conn, if_exists='replace', index=False)